## 1. 모델링

사전설정 (모델 훈련과 예측 모두 공통)

In [ ]:
import os

model_save_path = "saved_model2"
model_resi_save_path = "{}/residual_model".format(model_save_path)
# cols = ["KOSPI", "KOSPI_VOL", "CR", "GOLD", "NASDAQ", "DOW", "NIKKEI", "SHANGHAI", "INDI", "FOREIGN", "ORG"]
# cols = ["KOSPI", "KOSPI_VOL", "CR", "GOLD", "NASDAQ", "DOW", "NIKKEI", "SHANGHAI"]
cols = ["KOSPI", "KOSPI_HIGH", "KOSPI_LOW"]
len_x_ARMA = 30
len_y_nextday = 20
scale_method = "norm"

# 패키지 캐시 모두 지우기 (tf 등 이전에 남은 가비지가 영향을 미칠 수 있음)
for dir_ in ["/Users/hrdkdh/Documents/GitHub/predict/", "/Users/hrdkdh/venv/"]:
    os.system("find {} | grep -E '(__pycache__|\.pyc$|\pyo$)' | xargs rm -rf".format(dir_))

1-1) 데이터 크롤링

In [ ]:
from kospi_predict import Crawler

crawler = Crawler(crawl_page_max=30, perPage=100)
# crawler.crawlData(cols, save=True)
crawler.loadFromSavedFile(cols)
df_crawled = crawler.removeNan()
df_crawled = df_crawled.loc[:2000]
df_crawled

1-2. 데이터 전처리

In [ ]:
from kospi_predict import DataPreprocessor

dpp = DataPreprocessor(df_crawled, cols, scale_method, model_save_path)
dpp.sortByDate()
dpp.makeDiffRatio()
dpp.scalingForPredict()
dpp.makeAR(0, len_x_ARMA)
dpp.makeMA(2, len_x_ARMA)
dpp.makeTargetYs(len_y_nextday)
dpp.cutoffData(len_x_ARMA, len_y_nextday)

In [ ]:
#저장된 x_cols_info.txt 파일로 drop 하기
import json

x_cols_str = open("kospi_predictor_model/{}/x_cols_info.txt".format(model_save_path), "r").readlines()[0]
x_cols = json.loads(x_cols_str)
remove_cols = [x for x in dpp.df.columns.to_list() if "X_" in x and x not in x_cols]
dpp.df.drop(columns=remove_cols, inplace=True)

In [ ]:
dpp.splitData()
# dpp.df.to_excel("processed_data.xlsx", index=False)
dpp.df

1-3) 예측 모델로 추론한 후 예측 결과값을 dpp.df에 다시 concat

In [ ]:
from kospi_predict import Predictor

#dpp.df 행별로 20일치를 예측하고 concat해야 함

## 2. 예측값을 새로운 X변수로 하여 새로운 모델을 생성

2-1) 모델링

In [ ]:
#

2-2) 모델 검증하여 에러값을 확인, 잔차가 더 줄었는지 확인한다.

In [ ]:
#

2-3) 최신 데이터로 예측

In [ ]:
#최근일 기준으로 20일 뒤까지 예측하고 시각화


In [ ]:
#실제 기록과 예측값을 비교하여 예측이 얼마나 잘 맞는지 확인한다


In [ ]:
#모델을 tensorflow js 파일로 변환하여 저장
